<a href="https://www.bigdatauniversity.com"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/Images/CCLog.png" width=300, align="center"></a>

<h1 align=center><font size = 5>Capstone Project - Week 3 Clustering</font></h1>

## Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and clean data

In [1]:
# Import all required packages
import pandas as pd
import sklearn
import lxml
# Make sure lxml is installed in your conda environment package. Otw, error when read_html

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# Gets a list of tables from html
table_list = pd.read_html(link)

In [4]:
# Retrieve the first table on the list - Desired DataFrame
toronto_df = table_list[0]
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop rows (idx) that have BOROUGH has 'Not assigned'

In [5]:
idx = toronto_df.index[toronto_df['Borough'] == 'Not assigned'].tolist()
toronto_filter_df = toronto_df.drop(labels=idx)

In [6]:
toronto_filter_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### If NEIGHBOURHOOD has 'Not assigned' and BOROUGH has a location name assigned, then NEIGHBOURHOOD = BOROUGH

In [7]:
# Returns a list of index that meet requirement (only can use one CONDITIONS?) - can use multiple conditions by first building the BOOLEAN series and then passing it to 'index'
idx = toronto_filter_df.index[toronto_filter_df['Neighbourhood'] == 'Not assigned'].tolist()

In [8]:
# To change values for each index where the condition is met
for index in idx:
    toronto_filter_df.at[index, 'Neighbourhood'] = toronto_filter_df.at[index, 'Borough']

### More than one neighborhood can exist in one postal code area. Merge the names of the two neighbourhoods

In [9]:
toronto_filter_grouped_df = toronto_filter_df.copy()

In [10]:
toronto_filter_grouped_df['Neighbourhood'] = toronto_filter_df.groupby(by=['Postcode'])['Neighbourhood'].transform(lambda x: ", ".join(x))

In [11]:
toronto_filter_grouped_df.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"


### SCRAPED AND FILTERED DATAFRAME OF List_of_postal_codes_of_Canada

In [12]:
toronto_filter_grouped_df.drop_duplicates(inplace=True)
toronto_filter_grouped_df.head(11)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


### Finally, get the shape of the Dataframe

In [13]:
toronto_filter_grouped_df.shape

(103, 3)

## Get the latitude, longitude of the Postal Code

### Load CSV file that contains the geographical coordinates of each postal code (obtain from https://geocoder.readthedocs.io/index.html)

In [14]:
coord_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [15]:
df_test = pd.merge(toronto_filter_grouped_df, coord_df, how='inner', left_on='Postcode', right_on='Postal Code')
df_test.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


## Testing Out Different Clustering Algorithms on Grouping Neighbourhoods in their respective Boroughs

Based on this counts, we can how many unique postal codes are there in each Borough. There are 11 unique boroughs

In [16]:
import folium

In [17]:
df_test['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

## Using FourSquare API

### Define Foursquare Credentials and Version

In [18]:
import requests # library to handle requests
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [19]:
CLIENT_ID = 'O1PA0ORIUKGKJNDE1OVKH3WKAGGI05CS0IRZSKKWHOHSBJYZ' # your Foursquare ID
CLIENT_SECRET = 'LQIJ24FLLFAOEXXUFDXBJZBNP2ZELO4AQ4SPJEQUWJAHE1ER' # your Foursquare Secret
VERSION = '20190810'    # Simply a Date?
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1PA0ORIUKGKJNDE1OVKH3WKAGGI05CS0IRZSKKWHOHSBJYZ
CLIENT_SECRET:LQIJ24FLLFAOEXXUFDXBJZBNP2ZELO4AQ4SPJEQUWJAHE1ER


### Get all Asian Restaurants in neighbourhoods of North York

In [20]:
def getNearbyCategoryRestaurantVenues(names, latitudes, longitudes, radius=500, 
                                   CATEGORY_ID='4bf58dd8d48988d142941735', search_query="Asian"):
    # By default, CATEGORY_ID is set to Asian Restaurant 4bf58dd8d48988d142941735, American Restaurant = 4bf58dd8d48988d14e941735, search_query
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CATEGORY_ID,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        try:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                v['categories'][0]['name']) for v in results])
        except:
            print(name + ": Error")

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
idx = df_test.index[df_test['Borough'] == 'North York']
north_york_data = df_test.loc[idx]

In [22]:
north_york_asian_restaurant_venues = getNearbyCategoryRestaurantVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview West
Downsview, North Park, Upwood Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


### Explore Data to see what the unknown categories are and can be summarize as the specific nationality categories.

In [23]:
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Asian Restaurant         17
Japanese Restaurant      16
Chinese Restaurant       15
Sushi Restaurant         14
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           4
Ramen Restaurant          4
Dim Sum Restaurant        2
Filipino Restaurant       2
Restaurant                2
Caribbean Restaurant      1
Taiwanese Restaurant      1
Indonesian Restaurant     1
Noodle House              1
Name: Venue Category, dtype: int64

#### Ice Cream Shop - Japanese

In [24]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Italian Restaurant']['Venue'].value_counts()

Series([], Name: Venue, dtype: int64)

#### Sandwich Place - Vietnamese

In [25]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Sandwich Place']['Venue'].value_counts()

Series([], Name: Venue, dtype: int64)

#### Restaurant - Thai

In [26]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Asian Restaurant']['Venue'].value_counts()

Spring Rolls                                1
Restaurant At Novotel North York Toronto    1
one sit one seat teashop                    1
임가네                                         1
Yogi Noodle Delight                         1
Genghis Khan Mongolian Grill                1
Chili Secrets                               1
Mac Sushi                                   1
Li Cheng Restaurant                         1
Szechuan & Thai Specialty food              1
Wasabi Grill And Noodle                     1
luxury street Viennese                      1
Pho 88 發發餐廳                                 1
Miyako sushi                                1
Wasabi Grand Buffet                         1
Teriyaki Experience                         1
Casa Manila                                 1
Name: Venue, dtype: int64

#### Restaurant: 1 is Thai, 1 is Japanese

In [27]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Restaurant']['Venue'].value_counts()

Thai Express    1
Sushi Shop      1
Name: Venue, dtype: int64

#### Noodle House - Japanese

In [28]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Noodle House']['Venue'].value_counts()

Yunshang Rice Noodle（YF）云尚米线    1
Name: Venue, dtype: int64

#### Dumpling House

In [29]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Dumpling Restaurant']['Venue'].value_counts()

Series([], Name: Venue, dtype: int64)

In [30]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Ramen Restaurant']['Venue'].value_counts()

Hakata Shoryuken Ramen    1
Konjiki Ramen             1
KINTON RAMEN              1
Ajisen Ramen 味千ラーメン       1
Name: Venue, dtype: int64

#### All Sushi & Ramen Restaurant are Japanese Style Restaurants

In [31]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Ramen Restaurant', 'Venue Category'] = "Japanese Restaurant"
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Sushi Restaurant', 'Venue Category'] = "Japanese Restaurant"

In [32]:
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      34
Asian Restaurant         17
Chinese Restaurant       15
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           4
Dim Sum Restaurant        2
Filipino Restaurant       2
Restaurant                2
Taiwanese Restaurant      1
Caribbean Restaurant      1
Indonesian Restaurant     1
Noodle House              1
Name: Venue Category, dtype: int64

#### Dim Sum & Taiwanese are Chinese Style Restaurants

In [33]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Dim Sum Restaurant', 'Venue Category'] = "Chinese Restaurant"
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Taiwanese Restaurant', 'Venue Category'] = "Chinese Restaurant"

In [34]:
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      34
Chinese Restaurant       18
Asian Restaurant         17
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           4
Filipino Restaurant       2
Restaurant                2
Caribbean Restaurant      1
Indonesian Restaurant     1
Noodle House              1
Name: Venue Category, dtype: int64

#### Noodle House = Yunshang Rice Noodle（YF）云尚米线 - Added to Chinese Restaurant

In [35]:
north_york_asian_restaurant_venues.loc[north_york_asian_restaurant_venues['Venue Category'] == 'Noodle House', 'Venue Category'] = "Chinese Restaurant"

In [36]:
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      34
Chinese Restaurant       19
Asian Restaurant         17
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           4
Filipino Restaurant       2
Restaurant                2
Caribbean Restaurant      1
Indonesian Restaurant     1
Name: Venue Category, dtype: int64

#### Restaurant = Has 2, one is Thai Express which is a Thai Restaurant, one is Sushi Shop which is a Japanese Restaurant

In [37]:
idx = north_york_asian_restaurant_venues.index[north_york_asian_restaurant_venues['Venue Category'] == 'Restaurant']
idx

Int64Index([22, 32], dtype='int64')

In [38]:
for index in idx:
    if north_york_asian_restaurant_venues.at[index,'Venue'] == 'Thai Express':
        north_york_asian_restaurant_venues.at[index,'Venue Category'] = 'Thai Restaurant'
    else:
        north_york_asian_restaurant_venues.at[index,'Venue Category'] = 'Japanese Restaurant'

In [39]:
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      35
Chinese Restaurant       19
Asian Restaurant         17
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           5
Filipino Restaurant       2
Caribbean Restaurant      1
Indonesian Restaurant     1
Name: Venue Category, dtype: int64

#### Delete Asian Restaurant category, there's a mix of different nationality type cuisines. 

In [40]:
idx = north_york_asian_restaurant_venues.index[north_york_asian_restaurant_venues['Venue Category'] == 'Asian Restaurant']
idx

Int64Index([8, 9, 12, 15, 24, 26, 28, 34, 43, 44, 47, 56, 60, 86, 88, 98, 99], dtype='int64')

In [41]:
north_york_asian_restaurant_venues.drop(index=idx,inplace=True)
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      35
Chinese Restaurant       19
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           5
Filipino Restaurant       2
Caribbean Restaurant      1
Indonesian Restaurant     1
Name: Venue Category, dtype: int64

#### Delete the Carribean Restaurant as we are focused on Asian Cuisines

In [42]:
idx = north_york_asian_restaurant_venues.index[north_york_asian_restaurant_venues['Venue Category'] == 'Caribbean Restaurant']
north_york_asian_restaurant_venues.drop(index=idx,inplace=True)
north_york_asian_restaurant_venues['Venue Category'].value_counts()

Japanese Restaurant      35
Chinese Restaurant       19
Korean Restaurant        14
Vietnamese Restaurant     7
Thai Restaurant           5
Filipino Restaurant       2
Indonesian Restaurant     1
Name: Venue Category, dtype: int64

In [43]:
north_york_asian_restaurant_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
1,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Seoul House,43.719902,-79.457832,Korean Restaurant
2,Don Mills North,43.745906,-79.352188,Yummy BBQ,43.748270,-79.348580,Korean Restaurant
3,Don Mills North,43.745906,-79.352188,Gonoe Sushi,43.745737,-79.345991,Japanese Restaurant
4,Don Mills North,43.745906,-79.352188,Matsuda Japanese Cuisine & Teppanyaki,43.745494,-79.345821,Japanese Restaurant
5,Glencairn,43.709577,-79.445073,Miyako Sushi Restaurant,43.709111,-79.443930,Japanese Restaurant
6,Glencairn,43.709577,-79.445073,Kokoro Sushi,43.709031,-79.444053,Japanese Restaurant
7,Glencairn,43.709577,-79.445073,Tambayan,43.709071,-79.443451,Filipino Restaurant
10,Glencairn,43.709577,-79.445073,Lechon Republik,43.706164,-79.442423,Filipino Restaurant
11,"Flemingdon Park, Don Mills South",43.725900,-79.340923,Asian Legend,43.726591,-79.342188,Chinese Restaurant


### Now that we have our 'filtered' data, do some analysis on that data

#### Count the unique amount of 'Venue Category' for each neighborhood 

In [44]:
# one hot encoding
nyork_unique_asian_rest_onehot = pd.get_dummies(north_york_asian_restaurant_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyork_unique_asian_rest_onehot['Neighborhood'] = north_york_asian_restaurant_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyork_unique_asian_rest_onehot.columns[-1]] + list(nyork_unique_asian_rest_onehot.columns[:-1])
nyork_unique_asian_rest_onehot = nyork_unique_asian_rest_onehot[fixed_columns]

nyork_unique_asian_rest_onehot.head()

,Neighborhood,Chinese Restaurant,Filipino Restaurant,Indonesian Restaurant,Japanese Restaurant,Korean Restaurant,Thai Restaurant,Vietnamese Restaurant
0,"Lawrence Heights, Lawrence Manor",0,0,0,0,0,0,1
1,"Lawrence Heights, Lawrence Manor",0,0,0,0,1,0,0
2,Don Mills North,0,0,0,0,1,0,0
3,Don Mills North,0,0,0,1,0,0,0
4,Don Mills North,0,0,0,1,0,0,0


In [45]:
nyork_unique_asian_rest_grouped = nyork_unique_asian_rest_onehot.groupby('Neighborhood').mean().reset_index()
nyork_unique_asian_rest_grouped

,Neighborhood,Chinese Restaurant,Filipino Restaurant,Indonesian Restaurant,Japanese Restaurant,Korean Restaurant,Thai Restaurant,Vietnamese Restaurant
0,"Bathurst Manor, Downsview North, Wilson Heights",0.333333,0.0,0.000000,0.333333,0.333333,0.000000,0.000000
1,Bayview Village,0.500000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.625000,0.000000,0.250000,0.125000
3,Don Mills North,0.000000,0.0,0.000000,0.666667,0.333333,0.000000,0.000000
4,Downsview Northwest,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000
6,"Fairview, Henry Farm, Oriole",0.444444,0.0,0.000000,0.444444,0.000000,0.111111,0.000000
7,"Flemingdon Park, Don Mills South",0.333333,0.0,0.000000,0.666667,0.000000,0.000000,0.000000
8,Glencairn,0.000000,0.5,0.000000,0.500000,0.000000,0.000000,0.000000
9,"Lawrence Heights, Lawrence Manor",0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.500000


### Cluster Neighborhoods based on the frequency of types of Asian Restaurants today.

In [46]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [47]:
# set number of clusters -> Chinese, Japanese, Korean, South East Asian
kclusters = 4

nyork_unique_asian_rest_grouped_clustering = nyork_unique_asian_rest_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyork_unique_asian_rest_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 2, 2, 3, 0, 1, 1, 2, 0, 2, 1])

In [48]:
nyork_unique_asian_rest_grouped['Cluster Labels'] = kmeans.labels_

In [49]:
nyork_unique_asian_rest_grouped

,Neighborhood,Chinese Restaurant,Filipino Restaurant,Indonesian Restaurant,Japanese Restaurant,Korean Restaurant,Thai Restaurant,Vietnamese Restaurant,Cluster Labels
0,"Bathurst Manor, Downsview North, Wilson Heights",0.333333,0.0,0.000000,0.333333,0.333333,0.000000,0.000000,1
1,Bayview Village,0.500000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,1
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.625000,0.000000,0.250000,0.125000,2
3,Don Mills North,0.000000,0.0,0.000000,0.666667,0.333333,0.000000,0.000000,2
4,Downsview Northwest,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3
5,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0
6,"Fairview, Henry Farm, Oriole",0.444444,0.0,0.000000,0.444444,0.000000,0.111111,0.000000,1
7,"Flemingdon Park, Don Mills South",0.333333,0.0,0.000000,0.666667,0.000000,0.000000,0.000000,1
8,Glencairn,0.000000,0.5,0.000000,0.500000,0.000000,0.000000,0.000000,2
9,"Lawrence Heights, Lawrence Manor",0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.500000,0


In [50]:
nyork_unique_asian_rest_grouped.groupby('Cluster Labels').mean()

,Chinese Restaurant,Filipino Restaurant,Indonesian Restaurant,Japanese Restaurant,Korean Restaurant,Thai Restaurant,Vietnamese Restaurant
Cluster Labels,,,,,,,
0,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.750000
1,0.369591,0.000,0.005263,0.462573,0.124561,0.032749,0.005263
2,0.050000,0.125,0.000000,0.547917,0.083333,0.062500,0.131250
3,1.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
